In [0]:
declare or replace qry_str string;

create catalog if not exists identifier(:tgt_catalog);
use catalog identifier(:tgt_catalog);
create schema if not exists identifier(:tgt_schema);
use schema identifier(:tgt_schema);

-- Create sink table using schema inferred from foreign table
set var qry_str = 
  "create table if not exists " || :tgt_table || " as "
  "select " || :select_list || " from " || :src_catalog || '.' || :src_schema || '.' || :src_table || " "  
  "where 1 = 0";

execute immediate qry_str;

In [0]:
-- Enable liquid clustering if clustering columns are specified
begin
  declare qry_str string;

  if :sink_cluster_cols != '' then
    set qry_str = 
      "alter table " || :tgt_catalog || '.' || :tgt_schema || '.' || :tgt_table || " cluster by (" || :sink_cluster_cols || ")";
    execute immediate qry_str;
  end if;
end

In [0]:
-- Enable Iceberg reads
-- https://docs.databricks.com/aws/en/delta/uniform#enable-iceberg-reads-on-an-existing-table
begin
  declare qry_str string;
  declare enable_iceberg_reads boolean;
  set enable_iceberg_reads = cast(:enable_iceberg_reads as boolean);

  if enable_iceberg_reads is true then
    -- Disable deletion vectors
    set qry_str =
      'alter table ' || :tgt_catalog || '.' || :tgt_schema || '.' || :tgt_table || ' set tblproperties(\'delta.enableDeletionVectors\' = false);';
    execute immediate qry_str;
    
    -- Enable Iceberg reads
    set qry_str = 
      'alter table ' || :tgt_catalog || '.' || :tgt_schema || '.' || :tgt_table || ' set tblproperties('
        '\'delta.columnMapping.mode\' = \'name\','
        '\'delta.enableIcebergCompatV2\' = \'true\','
        '\'delta.universalFormat.enabledFormats\' = \'iceberg\')';
    execute immediate qry_str;
  end if;
end